In [84]:
import numpy as np
import networkx as nx
import sympy as sp

In [85]:
circuito = nx.Graph()

# Aggiunta dei nodi (punti di connessione) 10M
circuito.add_nodes_from(["A", "B", "C", "D","E"])  # Nodi del circuito
R1=198.8
R2=1193
R3=297
R4= (R2/R1)*R3
R5=10 * 10**9
R0=0.1
resistenze = {
    ("A","B"): R1,
    ("A","C"): R3,
    ("C","B"): R5,
    ("B","D"): R2,
    ("C","D"): R4,     
    ("D","E"): R0
     
}


for (n1, n2), r in resistenze.items():
    circuito.add_edge(n1, n2, resistance=r)

generatori = {
    "A": 3,  
    
    "E":0   
}
nodi_variabili = [n for n in circuito.nodes if n not in generatori]
print(nodi_variabili)

['B', 'C', 'D']


In [86]:
n = len(nodi_variabili)
Y = np.zeros((n, n))
I = np.zeros(n)


#costruzione della matrice
for i, nodo in enumerate(nodi_variabili):
    print(i,nodo)
    somma_ammettenze = 0
    
    for vicino in circuito.neighbors(nodo):
        R = circuito[nodo][vicino]['resistance']
        G = 1 / R  # Ammettenza G = 1/R
        
        if vicino in nodi_variabili:
            j = nodi_variabili.index(vicino)
            Y[i, j] -= G  # Termini fuori diagonale
        
        elif vicino in generatori:
            I[i] += G * generatori[vicino]  # Termini di corrente
        
        somma_ammettenze += G
    
    Y[i, i] = somma_ammettenze 
#tensioni
print(Y)
print(I)
V = np.linalg.solve(Y, I)

tensioni_nodi = {n: V[i] for i, n in enumerate(nodi_variabili)}
tensioni_nodi.update(generatori) 

0 B
1 C
2 D
[[ 5.86840415e-03 -1.00000000e-10 -8.38222967e-04]
 [-1.00000000e-10  3.92807662e-03 -5.61073151e-04]
 [-8.38222967e-04 -5.61073151e-04  1.00013993e+01]]
[0.01509054 0.01010101 0.        ]


In [87]:
correnti = {}
for (n1, n2), r in resistenze.items():
    V1, V2 = tensioni_nodi[n1], tensioni_nodi[n2]
    I = (V1 - V2) / r  # Ohm: V = IR ⟹ I = V/R
    correnti[(n1, n2)] = I



In [88]:
# Stampa dei risultati
print("Potenziali ai nodi:")
for nodo, v in tensioni_nodi.items():
    print(f"  Nodo {nodo}: {v:.3f} V")
dV=tensioni_nodi[n1]-tensioni_nodi[n2]
print("\nCorrenti nei rami:")
for (n1, n2), i in correnti.items():
    print(f"  Corrente tra {n1} e {n2}: {i:.3f} A")


Potenziali ai nodi:
  Nodo B: 2.572 V
  Nodo C: 2.572 V
  Nodo D: 0.000 V
  Nodo A: 3.000 V
  Nodo E: 0.000 V

Correnti nei rami:
  Corrente tra A e B: 0.002 A
  Corrente tra A e C: 0.001 A
  Corrente tra C e B: 0.000 A
  Corrente tra B e D: 0.002 A
  Corrente tra C e D: 0.001 A
  Corrente tra D e E: 0.004 A


In [89]:
G0=sp.symbols('G0')
G1=sp.symbols('G1')
G2=sp.symbols('G2')
G3=sp.symbols('G3')
G4=sp.symbols('G4')
G5=sp.symbols('G5')
circuito = nx.Graph()
resistenze = {
    ("A","B"): G1,
    ("A","C"): G3,
    ("B","D"): G2,
    ("C","D"): G4,
    ("D","E"): G0,
    ("B","C"): G5     
}
for (n1, n2), r in resistenze.items():
    circuito.add_edge(n1, n2, resistance=r)

generatori = {
    "A": 22.6,  
    
    "E":0   
}
nodi_variabili = [n for n in circuito.nodes if n not in generatori]
print(nodi_variabili)
print(-1 * G1)


['B', 'C', 'D']
-G1


In [90]:
import sympy as sp
import networkx as nx

# Definizione dei simboli per le conduttanze
G0, G1, G2, G3, G4, G5 = sp.symbols('G0 G1 G2 G3 G4 G5')

# Creazione del grafo con conduttanze
circuito = nx.Graph()
conduttanze = {
    ("A", "B"): G1,
    ("A", "C"): G3,
    ("B", "D"): G2,
    ("C", "D"): G4,
    ("D", "E"): G0,
    ("B", "C"): G5     
}

# Aggiunta degli archi al grafo con attributo "conductance"
for (n1, n2), G in conduttanze.items():
    circuito.add_edge(n1, n2, conductance=G)

# Definizione dei generatori di tensione
generatori = {
    "A": 22.6,  
    "E": 0   
}

# Troviamo i nodi variabili (quelli che non hanno un generatore associato)
nodi_variabili = [n for n in circuito.nodes if n not in generatori]

# Creazione della matrice simbolica di ammettenza Y
n = len(nodi_variabili)
Y = sp.zeros(n, n)  # Matrice simbolica inizializzata a 0

for i, nodo in enumerate(nodi_variabili):
    somma_ammettenze = 0  # Per il termine diagonale
    
    for vicino in circuito.neighbors(nodo):
        G = circuito[nodo][vicino]['conductance']  # Prendiamo la conduttanza assegnata al ramo
        
        if vicino in nodi_variabili:
            j = nodi_variabili.index(vicino)
            Y[i, j] = -G  # Termini fuori diagonale
        
        somma_ammettenze += G  # Somma delle conduttanze per il termine diagonale
    
    Y[i, i] = somma_ammettenze  # Termine diagonale

# Stampiamo la matrice simbolica di ammettenza
print("Matrice di ammettenza Y:")
sp.pprint(Y)

# Calcoliamo il determinante simbolico di Y
det_Y = Y.det()
print("\nDeterminante simbolico della matrice di ammettenza:")
print(det_Y)


Matrice di ammettenza Y:
⎡G₁ + G₂ + G₅      -G₅           -G₂     ⎤
⎢                                        ⎥
⎢    -G₅       G₃ + G₄ + G₅      -G₄     ⎥
⎢                                        ⎥
⎣    -G₂           -G₄       G₀ + G₂ + G₄⎦

Determinante simbolico della matrice di ammettenza:
G0*G1*G3 + G0*G1*G4 + G0*G1*G5 + G0*G2*G3 + G0*G2*G4 + G0*G2*G5 + G0*G3*G5 + G0*G4*G5 + G1*G2*G3 + G1*G2*G4 + G1*G2*G5 + G1*G3*G4 + G1*G4*G5 + G2*G3*G4 + G2*G3*G5 + G3*G4*G5


In [91]:
print(dV)
dv= 100 * 10 ** -6
us=(R2/R1) * (268-R3) * dv / 3 ** (0.5)
print(us)

0.0003597844660287172
-0.010047579106214025
